# Approximate steady state error

We approximate the maximum error due to the steady state assumptions by following a 9 step work flow.
The theory behind this approach is laied out in Appendix C of the paper.

 1. draw initial demand
 2. solve state equations (steady state)
 3. modify demand (i.e. modify mass flows at demand)
 4. solve hydraulic system with new mass flows
 5. solve thermal system with new mass flows
 6. solve steady state with new demands
 7. compare thermal solution (5) and steady-state solution (6); i.e., compare temperatures at demands
 8. store results over multiple runs and calculate means and variances



### Imports:

In [1]:
import numpy as np
import tensorflow as tf
from libdnnmcmc.utility import load_scenario
from libdnnmcmc.steady_state_solvers import SE_solver, my_fixpoint_iteratror

SE, VM, d_prior_dist, _ = load_scenario('loop')
cycles = VM.cycles
FI = my_fixpoint_iteratror()

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


### settings:

In [2]:
n_runs = 50
q_modifier = [0.7, 1.3]

### setup:

In [3]:
n_modifier = len(q_modifier)
# init arrays to store results:
z_pattern = np.zeros((n_runs, n_modifier))
Ta_ss, Tb_ss, Tc_ss, Td_ss = np.zeros_like(z_pattern), np.zeros_like(z_pattern), np.zeros_like(z_pattern), np.zeros_like(z_pattern)
Ta_qss, Tb_qss, Tc_qss, Td_qss = np.zeros_like(z_pattern), np.zeros_like(z_pattern), np.zeros_like(z_pattern), np.zeros_like(z_pattern)

# define wrapper around solver:
solve_SE = SE_solver()
def _solve_SE(demands, SE):
    for dem in SE.demands.keys():
        ind = SE.dem_ind[dem]
        d = demands[ind]
        SE.Q_heat[ind].assign(d)
    solve_SE(SE, cycles)
    return tf.reduce_sum(SE.evaluate_state_equations('forwardpass')**2)

### Analysis:

In [4]:
# 1 draw inital demand:
q_init = d_prior_dist.sample(n_runs)

for mod_ind, modifier in enumerate(q_modifier):
    q_new = q_init * modifier

    for run_ind in range(n_runs):

        # 2. solve state equations:
        psi = _solve_SE(q_init[run_ind, :], SE)

        # if the calculations did not converge reset the internal state of the SE object and try again
        if psi > 1.e-5:
            SE.set_init_state()
            psi = _solve_SE(q_init[run_ind, :], SE)
            if psi > 1.e-5:
                raise Exception(f'System did not converge with psi = {psi:4f}')

        # 3. modify mass flows according to demand:
        mf_init = tf.concat([SE.mf[SE.find_edge[k]] for k in SE.demands.keys()], axis=0)
        mf_new = mf_init * modifier

        #  4. solve hydraulic system with new mass flows
        FI.solve_mf(SE, mf_new, cycles)

        #  5. solve thermal system with new mass flows
        FI.solve_temp(SE)
        TA_qss = SE.T[SE.find_node['BS']]
        TB_qss = SE.T[SE.find_node['FS']]
        TC_qss = SE.T[SE.find_node['ES']]
        TD_qss = SE.T[SE.find_node['IS']]
        Ta_qss [run_ind, mod_ind], Tb_qss[run_ind, mod_ind], Tc_qss[run_ind, mod_ind], Td_qss[run_ind, mod_ind] = TA_qss, TB_qss, TC_qss, TD_qss

        #  6. solve steady state with new demands
        _solve_SE(q_new[run_ind,:], SE)

        #  7. compare thermal solution and steady-state solution
        TA_ss = SE.T[SE.find_node['BS']]
        TB_ss = SE.T[SE.find_node['FS']]
        TC_ss = SE.T[SE.find_node['ES']]
        TD_ss = SE.T[SE.find_node['IS']]

        # broadcast differences:
        print(f'T_diff Dem A: {TA_ss - TA_qss} \n'
              f'T_diff Dem B: {TB_ss - TB_qss} \n'
              f'T_diff Dem c: {TC_ss - TC_qss} \n'
              f'T_diff Dem D: {TD_ss - TD_qss} \n')

        #  8. save results for further analysis
        Ta_ss [run_ind, mod_ind], Tb_ss[run_ind, mod_ind], Tc_ss[run_ind, mod_ind], Td_ss[run_ind, mod_ind] = TA_ss, TB_ss, TC_ss, TD_ss

T_diff Dem A: [0.16886726] 
 T_diff Dem B: [1.74257629] 
 T_diff Dem c: [0.05077148] 
 T_diff Dem D: [0.16594071] 
 
T_diff Dem A: [0.20352625] 
 T_diff Dem B: [0.49475922] 
 T_diff Dem c: [0.04029388] 
 T_diff Dem D: [0.19316898] 
 
T_diff Dem A: [0.30564609] 
 T_diff Dem B: [0.97170311] 
 T_diff Dem c: [0.03681089] 
 T_diff Dem D: [0.19846425] 
 
T_diff Dem A: [0.16090781] 
 T_diff Dem B: [0.87829356] 
 T_diff Dem c: [0.04576766] 
 T_diff Dem D: [0.62237516] 
 
T_diff Dem A: [0.21125695] 
 T_diff Dem B: [-3.57838673] 
 T_diff Dem c: [0.05206049] 
 T_diff Dem D: [1.60035737] 
 
T_diff Dem A: [0.24622681] 
 T_diff Dem B: [0.71053122] 
 T_diff Dem c: [0.0362094] 
 T_diff Dem D: [0.13871323] 
 
T_diff Dem A: [0.28344678] 
 T_diff Dem B: [0.98361553] 
 T_diff Dem c: [0.03814324] 
 T_diff Dem D: [0.15163684] 
 
T_diff Dem A: [0.28553158] 
 T_diff Dem B: [0.75545733] 
 T_diff Dem c: [0.04720897] 
 T_diff Dem D: [0.16853765] 
 
T_diff Dem A: [0.1634733] 
 T_diff Dem B: [1.15117604] 
 T_diff 

In [5]:
# 8. calculate mean deviations and deviation variances
print(f'########################## probabilistic comaprison: ############################## \n')
for mod_ind, mod in enumerate(q_modifier):
    # 10 compare results over multiple runs:
    diff_a = Ta_ss[:, mod_ind] - Ta_qss[:, mod_ind]
    diff_b = Tb_ss[:, mod_ind] - Tb_qss[:, mod_ind]
    diff_c = Tc_ss[:, mod_ind] - Tc_qss[:, mod_ind]
    diff_d = Td_ss[:, mod_ind] - Td_qss[:, mod_ind]
    print(f'q_modifier: {mod} \n'
          f'T_diff Dem A: mean: {np.mean(np.abs(diff_a)):2f}, std: {np.std(diff_a):2f} \n'
          f'T_diff Dem B: mean: {np.mean(np.abs(diff_b)):2f}, std: {np.std(diff_b):2f} \n'
          f'T_diff Dem C: mean: {np.mean(np.abs(diff_c)):2f}, std: {np.std(diff_c):2f} \n'
          f'T_diff Dem D: mean: {np.mean(np.abs(diff_d)):2f}, std: {np.std(diff_d):2f}')


########################## probabilistic comaprison: ############################## 

q_modifier: 0.7 
T_diff Dem A: mean: 0.320411, std: 0.278607 
T_diff Dem B: mean: 1.576392, std: 1.464184 
T_diff Dem C: mean: 0.042193, std: 0.010959 
T_diff Dem D: mean: 0.278834, std: 0.223719
q_modifier: 1.3 
T_diff Dem A: mean: 0.093414, std: 0.047194 
T_diff Dem B: mean: 0.617416, std: 0.657955 
T_diff Dem C: mean: 0.013152, std: 0.003579 
T_diff Dem D: mean: 0.085424, std: 0.048943
